## Query models

In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob

from moral_lens.dilemma import DilemmaRunner
from moral_lens.judge import JudgeRunner

In [ ]:
decision_model_ids = [
    # # # RQ1 # # #
    # "gpt-4.1-2025-04-14", # done 5
    # "gpt-4.1-mini-2025-04-14", # done 5
    # "gpt-4.1-nano-2025-04-14", # done 5
    # "gpt-4o-2024-11-20", # done 5
    # "gpt-4o-2024-08-06", # done 5
    # "gpt-4o-2024-05-13", # done 5
    # "gpt-4o-mini-2024-07-18", # done 5
    # "gpt-3.5-turbo-0125", # done 5
    # "gpt-3.5-turbo-1106", # done 5
    # "gpt-3.5-turbo-0613", # depricated

    # "google/gemini-2.5-flash-preview",
    # "google/gemini-2.0-flash-001", # done 5
    # "google/gemini-2.0-flash-lite-001", # done 5
    # "google/gemini-pro-1.5", # done 5
    # "google/gemini-flash-1.5", # done 5
    # "google/gemini-flash-1.5-8b", # done 5

    # "meta-llama/llama-4-maverick", # done 5
    # "meta-llama/llama-4-scout", # done 5
    # "meta-llama/llama-3.3-70b-instruct", # done 5
    # "meta-llama/llama-3.1-405b-instruct", # 83% approval rate
    # "meta-llama/llama-3.1-70b-instruct", # done 5
    # "meta-llama/llama-3.1-8b-instruct", # done 5
    # "meta-llama/llama-3-70b-instruct", # done 5
    # "meta-llama/llama-3-8b-instruct", # done 5
    # "meta-llama/llama-2-70b-chat", # no valid responses in early stages

    # "anthropic/claude-3.7-sonnet:beta",
    # "anthropic/claude-3.5-sonnet:beta",
    # "anthropic/claude-3.5-sonnet-20240620:beta",
    # "anthropic/claude-3.5-haiku:beta", # done 5
    # "anthropic/claude-3-sonnet:beta",
    # "anthropic/claude-3-haiku:beta", # 79% approval rate

    # "microsoft/phi-4", # done 5
    # "microsoft/phi-3.5-mini-128k-instruct",
    # "microsoft/phi-3-medium-128k-instruct", # 89% approval rate
    # "microsoft/phi-3-mini-128k-instruct", # 88% approval rate

    # "qwen/qwen-max", # done 5
    # "qwen/qwen-plus", # done 5
    # "qwen/qwen-turbo", # done 5

    # "qwen/qwen-2.5-72b-instruct", # done 5
    # "qwen/qwen-2.5-7b-instruct", # done 5
    # "qwen/qwen-2-72b-instruct", # done 5

    # "qwen/qwen3-32b:nothink",
    # "qwen/qwen3-32b:think",

    # "qwen/qwen3-30b-a3b:nothink",
    "qwen/qwen3-30b-a3b:think",

    # "deepseek/deepseek-chat", # done 5
    # "deepseek/deepseek-chat-v3-0324", # done 5
    # "deepseek/deepseek-prover-v2",

    # "google/gemma-3-27b-it", # done 5
    # "google/gemma-3-12b-it", # done 5
    # "google/gemma-3-4b-it", # done 5

    # "google/gemma-2-27b-it", # done 5
    # "google/gemma-2-9b-it", # done 5

    # "cohere/command-r", # rate limits
    # "cohere/command-r-plus", # haven't tried

    # "allenai/olmo-7b-instruct", # Have not tried a full run yet, but I suspect low approval rate

    # "mistralai/mixtral-8x22b-instruct", # low approval rate
    # "mistralai/mixtral-8x7b-instruct", # done 5
    # "mistralai/mistral-7b-instruct-v0.3", # done 5
    # "mistralai/mistral-7b-instruct-v0.1", # done 5
    # "mistralai/mistral-nemo", # done 5
    # "mistralai/mistral-small", # done 5
    # "mistralai/mistral-large",
    # "mistralai/mistral-large-2407",

    # "amazon/nova-micro-v1",
    # "amazon/nova-lite-v1",
    # "amazon/nova-pro-v1",


    # # # HuggingFace # # #
    # "google/gemma-3-1b-it",

    # # # RQ2 # # #
    # "OLMo-2-0325-32B-SFT",
    # "OLMo-2-0325-32B-DPO",
    # "OLMo-2-0325-32B-Instruct",

    # "OLMo-2-1124-13B-SFT",
    # "OLMo-2-1124-13B-DPO",
    # "OLMo-2-1124-13B-Instruct",

    # "Llama-3.1-Tulu-3-70B-SFT",
    # "Llama-3.1-Tulu-3-70B-DPO",
    # "Llama-3.1-Tulu-3-70B",

    # # # RQ3 # # #
    # "tulu-v2.5-dpo-13b-hh-rlhf-60k",
    # "tulu-v2.5-dpo-13b-chatbot-arena-2023",
    # "tulu-v2.5-dpo-13b-stackexchange-60k",
    # "tulu-v2.5-dpo-13b-nectar-60k",

    # "tulu-v2.5-ppo-13b-hh-rlhf-60k",
    # "tulu-v2.5-ppo-13b-chatbot-arena-2023",
    # "tulu-v2.5-ppo-13b-stackexchange-60k",
    # "tulu-v2.5-ppo-13b-nectar-60k",

    # "allenai/tulu-2-13b",

    # "allenai/tulu-v2.5-dpo-13b-uf-mean",
    # "allenai/tulu-v2.5-dpo-13b-helpsteer",
    # "allenai/tulu-v2.5-dpo-13b-shp2",
    # "allenai/tulu-v2.5-dpo-13b-stackexchange",
    # "allenai/tulu-v2.5-dpo-13b-uf-overall",
    # "allenai/tulu-v2.5-dpo-13b-capybara",
    # "allenai/tulu-v2.5-dpo-13b-prm-phase-2",
    # "allenai/tulu-v2.5-dpo-13b-hh-rlhf",
    # "allenai/tulu-v2.5-dpo-13b-nectar",
    # "allenai/tulu-v2.5-dpo-13b-chatbot-arena-2023",
    # "allenai/tulu-v2.5-dpo-13b-chatbot-arena-2024",
    # "allenai/tulu-v2.5-dpo-13b-alpacafarm-human-pref",
    # "allenai/tulu-v2.5-dpo-13b-alpacafarm-gpt4-pref",
    # "allenai/tulu-v2.5-dpo-13b-argilla-orca-pairs",

    # # RQ4 # # #
    # "qwen/qwq-32b", # done 5

    # "deepseek/deepseek-r1",
    # "deepseek/deepseek-r1-distill-qwen-1.5b",
    # "deepseek/deepseek-r1-distill-llama-8b", # done 5
    # "deepseek/deepseek-r1-distill-llama-70b", # done 5

    # "microsoft/phi-4-reasoning-plus",

    # "x-ai/grok-3-mini-beta", # done 5
]
judge_model_ids = [
    # "gpt-4.1-mini-2025-04-14",
    # "gpt-4o-2024-08-06",
    "google/gemini-2.5-flash-preview",
    # "gemini-2.5-flash-preview-04-17",
]
# RESULTS_DIR = "data/20250507/all_model_runs/"
# RESULTS_DIR = "data/20250507/test/"
RESULTS_DIR = "data/20250507/reasoning_model_runs/"

In [ ]:
exps = [
    "s1",
    "s2",
    "s3",
    "s4",
    "s5",
]
for decision_model_id in decision_model_ids:
    continue_to_next_model = False
    for exp in exps:
        dr = DilemmaRunner(
            model_id=decision_model_id,
            decision_run_name=exp,
            results_dir=RESULTS_DIR,
            # override_decision_temperature=1.0,  # default 0.0
            # prompts_template='reasoning_after',
            prompts_template='no_reasoning',
            choices_filename="choices_672.csv",
        )
        await dr.run(
            # limit=10,
            # disable_validation=True,
            try_retries=False
        )
        # dr.process()

        # valid_pct = (dr.data.raw_response.str.len() != 0).sum() / len(dr.data)
    #     if valid_pct < 0.85:
    #         print(f"Warning: {valid_pct:.2%} of responses for model {decision_model_id} are empty.")
    #         continue_to_next_model = True
    #     if continue_to_next_model:
    #         continue
    # if continue_to_next_model:
    #     print(f"Finished decision model {decision_model_id}.\n\n")
    #     continue
    print(f"Finished decision model {decision_model_id}.\n\n")

## Judge

In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob

from moral_lens.dilemma import DilemmaRunner
from moral_lens.judge import JudgeRunner

In [ ]:
decision_model_ids = [
    # # # RQ1 # # #
    # "gpt-4.1-2025-04-14", # done 5
    # "gpt-4.1-mini-2025-04-14", # done 5
    # "gpt-4.1-nano-2025-04-14", # done 5
    # "gpt-4o-2024-11-20", # done 5
    # "gpt-4o-2024-08-06", # done 5
    # "gpt-4o-2024-05-13", # done 5
    # "gpt-4o-mini-2024-07-18", # done 5
    # "gpt-3.5-turbo-0125", # done 5
    # "gpt-3.5-turbo-1106", # done 5

    # "google/gemini-2.5-flash-preview",
    # "google/gemini-2.0-flash-001", # done 5
    # "google/gemini-2.0-flash-lite-001", # done 5
    # "google/gemini-pro-1.5", # done 5
    # "google/gemini-flash-1.5", # done 5
    # "google/gemini-flash-1.5-8b", # done 5

    # "meta-llama/llama-4-maverick", # done 5
    # "meta-llama/llama-4-scout", # done 5
    # "meta-llama/llama-3.3-70b-instruct", # done 5
    # "meta-llama/Llama-3.2-3B-Instruct", # done 5
    # "meta-llama/Llama-3.2-1B-Instruct", # done 5
    # "meta-llama/llama-3.1-405b-instruct",
    # "meta-llama/llama-3.1-70b-instruct", # done 5
    # "meta-llama/llama-3.1-8b-instruct", # done 5
    # "meta-llama/llama-3-70b-instruct", # done 5
    # "meta-llama/llama-3-8b-instruct", # done 5

    # "anthropic/claude-3.7-sonnet:beta",
    # "anthropic/claude-3.5-sonnet:beta",
    # "anthropic/claude-3.5-sonnet-20240620:beta",
    # "anthropic/claude-3.5-haiku:beta",
    # "anthropic/claude-3-sonnet:beta",
    # "anthropic/claude-3-haiku:beta",

    # "microsoft/phi-4", # done 5
    # "microsoft/phi-3.5-mini-128k-instruct",
    # "microsoft/phi-3-medium-128k-instruct", # done 3
    # "microsoft/phi-3-mini-128k-instruct", # done 3

    # "qwen/qwen-turbo", # done 5
    # "qwen/qwen-plus", # done 5
    # "qwen/qwen-max", # done 5

    # "qwen/qwen-2.5-7b-instruct", # done 5
    # "qwen/qwen-2.5-72b-instruct", # done 5

    # "Qwen2.5-1.5B-Instruct",
    # "Qwen2.5-3B-Instruct",
    # "Qwen2.5-14B-Instruct",
    # "Qwen2.5-32B-Instruct",

    # "Qwen1.5-4B-Chat",
    # "Qwen1.5-7B-Chat",
    # "Qwen1.5-14B-Chat",
    # "Qwen1.5-32B-Chat",
    # "Qwen1.5-72B-Chat",

    # "deepseek/deepseek-chat", # done 5
    # "deepseek/deepseek-chat-v3-0324", # done 5

    # "google/gemma-3-1b-it", # done 5
    # "google/gemma-3-4b-it", # done 5
    # "google/gemma-3-12b-it", # done 5
    # "google/gemma-3-27b-it", # done 5

    # "mistralai/mixtral-8x7b-instruct", # done 5
    # "mistralai/mistral-7b-instruct-v0.3", # done 5
    # "mistralai/mistral-7b-instruct-v0.1", # done 5

    # # # RQ2 # # #
    # "OLMo-2-0325-32B-SFT", # done 3
    # "OLMo-2-0325-32B-DPO", # done 3
    # "OLMo-2-0325-32B-Instruct",

    # "OLMo-2-1124-13B-SFT", # done 3
    # "OLMo-2-1124-13B-DPO", # done 3
    # "OLMo-2-1124-13B-Instruct",

    # "Llama-3.1-Tulu-3-70B-SFT", # done 3
    # "Llama-3.1-Tulu-3-70B-DPO", # done 3
    # "Llama-3.1-Tulu-3-70B",

    # "amd/Instella-3B-SFT", # done 3
    # "amd/Instella-3B-Instruct", # done 3

    # # # RQ3 # # #
    # "tulu-v2.5-dpo-13b-hh-rlhf-60k",
    # "tulu-v2.5-dpo-13b-chatbot-arena-2023",
    # "tulu-v2.5-dpo-13b-stackexchange-60k",
    # "tulu-v2.5-dpo-13b-nectar-60k",

    # "tulu-v2.5-ppo-13b-hh-rlhf-60k",
    # "tulu-v2.5-ppo-13b-chatbot-arena-2023",
    # "tulu-v2.5-ppo-13b-stackexchange-60k",
    # "tulu-v2.5-ppo-13b-nectar-60k",

    # "allenai/tulu-2-13b", # done 3

    # "allenai/tulu-v2.5-dpo-13b-uf-mean",
    # "allenai/tulu-v2.5-dpo-13b-helpsteer",
    # "allenai/tulu-v2.5-dpo-13b-shp2",
    # "allenai/tulu-v2.5-dpo-13b-stackexchange",
    # "allenai/tulu-v2.5-dpo-13b-uf-overall",
    # "allenai/tulu-v2.5-dpo-13b-capybara",
    # "allenai/tulu-v2.5-dpo-13b-prm-phase-2",
    # "allenai/tulu-v2.5-dpo-13b-hh-rlhf",
    # "allenai/tulu-v2.5-dpo-13b-nectar",
    # "allenai/tulu-v2.5-dpo-13b-chatbot-arena-2023",
    # "allenai/tulu-v2.5-dpo-13b-chatbot-arena-2024",
    # "allenai/tulu-v2.5-dpo-13b-alpacafarm-human-pref",
    # "allenai/tulu-v2.5-dpo-13b-alpacafarm-gpt4-pref",
    # "allenai/tulu-v2.5-dpo-13b-argilla-orca-pairs",

    # # RQ4 # # #
    # "qwen/qwq-32b", # done 5

    # "deepseek/deepseek-r1",
    # "deepseek/deepseek-r1-distill-qwen-1.5b",
    # "deepseek/deepseek-r1-distill-llama-8b",
    # "deepseek/deepseek-r1-distill-llama-70b", # done 5

    # "microsoft/phi-4-reasoning-plus",

    # "x-ai/grok-3-mini-beta", # done 5
]
judge_model_ids = [
    # "gpt-4.1-mini-2025-04-14",
    # "gpt-4o-2024-08-06",
    "google/gemini-2.5-flash-preview",
    # "gemini-2.5-flash-preview-04-17",
]
RESULTS_DIR = "data/20250507/all_model_runs/"
# RESULTS_DIR = "data/20250507/dpo_model_runs/"
# RESULTS_DIR = "data/20250507/preference_runs/"
# RESULTS_DIR = "data/20250507/reasoning_model_runs/"

In [ ]:
exps = [
    "s1",
    "s2",
    "s3",
    "s4",
    "s5",
]
for decision_model_id in decision_model_ids:
    for exp in exps:
        jr = JudgeRunner(
            decision_model_id=decision_model_id,
            decision_run_name=exp,
            judge_model_id=judge_model_ids[0],
            judge_run_name=exp,
            results_dir=RESULTS_DIR,
            judge_cot=True,
            override_judge_temperature=0.0,
        )
        await jr.run_rationales()

    print(f"Finished analyzing decision model {decision_model_id}.\n\n")

## final qwen runs

In [ ]:
judge_model_ids = [
    # "gpt-4.1-mini-2025-04-14",
    # "gpt-4o-2024-08-06",
    "google/gemini-2.5-flash-preview",
    # "gemini-2.5-flash-preview-04-17",
]

In [ ]:
decision_model_ids = ["qwen/qwen3-30b-a3b:think"]
RESULTS_DIR = "data/20250507/reasoning_model_runs/"
exps = [
    "s1",
    "s2",
    "s3",
    "s4",
    "s5",
]
for decision_model_id in decision_model_ids:
    continue_to_next_model = False
    for exp in exps:
        dr = DilemmaRunner(
            model_id=decision_model_id,
            decision_run_name=exp,
            results_dir=RESULTS_DIR,
            # override_decision_temperature=1.0,  # default 0.0
            # prompts_template='reasoning_after',
            prompts_template='no_reasoning',
            choices_filename="choices_672.csv",
        )
        await dr.run(
            # limit=10,
            # disable_validation=True,
            try_retries=False
        )
    print(f"Finished decision model {decision_model_id}.\n\n")

for decision_model_id in decision_model_ids:
    for exp in exps:
        jr = JudgeRunner(
            decision_model_id=decision_model_id,
            decision_run_name=exp,
            judge_model_id=judge_model_ids[0],
            judge_run_name=exp,
            results_dir=RESULTS_DIR,
            judge_cot=True,
            override_judge_temperature=0.0,
        )
        await jr.run_rationales()

    print(f"Finished analyzing decision model {decision_model_id}.\n\n")

In [ ]:
decision_model_ids = ["qwen/qwen3-30b-a3b:nothink"]
RESULTS_DIR = "data/20250507/all_model_runs/"
exps = [
    "s1",
    "s2",
    "s3",
    "s4",
    "s5",
]
for decision_model_id in decision_model_ids:
    continue_to_next_model = False
    for exp in exps:
        dr = DilemmaRunner(
            model_id=decision_model_id,
            decision_run_name=exp,
            results_dir=RESULTS_DIR,
            override_decision_temperature=1.0,  # default 0.0
            # prompts_template='reasoning_after',
            # prompts_template='no_reasoning',
            choices_filename="choices_672.csv",
        )
        await dr.run(
            # limit=10,
            # disable_validation=True,
            try_retries=False
        )
    print(f"Finished decision model {decision_model_id}.\n\n")

for decision_model_id in decision_model_ids:
    for exp in exps:
        jr = JudgeRunner(
            decision_model_id=decision_model_id,
            decision_run_name=exp,
            judge_model_id=judge_model_ids[0],
            judge_run_name=exp,
            results_dir=RESULTS_DIR,
            judge_cot=True,
            override_judge_temperature=0.0,
        )
        await jr.run_rationales()

    print(f"Finished analyzing decision model {decision_model_id}.\n\n")

In [ ]:
decision_model_ids = ["qwen/qwen3-30b-a3b:nothink"]
RESULTS_DIR = "data/20250507/reasoning_after_runs/"
exps = [
    "after1",
    "after2",
    "after3",
    "after4",
    "after5",
]
for decision_model_id in decision_model_ids:
    continue_to_next_model = False
    for exp in exps:
        dr = DilemmaRunner(
            model_id=decision_model_id,
            decision_run_name=exp,
            results_dir=RESULTS_DIR,
            override_decision_temperature=1.0,  # default 0.0
            prompts_template='reasoning_after',
            # prompts_template='no_reasoning',
            choices_filename="choices_672.csv",
        )
        await dr.run(
            # limit=10,
            # disable_validation=True,
            try_retries=False
        )
    print(f"Finished decision model {decision_model_id}.\n\n")

for decision_model_id in decision_model_ids:
    for exp in exps:
        jr = JudgeRunner(
            decision_model_id=decision_model_id,
            decision_run_name=exp,
            judge_model_id=judge_model_ids[0],
            judge_run_name=exp,
            results_dir=RESULTS_DIR,
            judge_cot=True,
            override_judge_temperature=0.0,
        )
        await jr.run_rationales()

    print(f"Finished analyzing decision model {decision_model_id}.\n\n")

## Reasoning before vs after

In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob

from moral_lens.dilemma import DilemmaRunner
from moral_lens.judge import JudgeRunner

In [ ]:
decision_model_ids = [
    # "gpt-4.1-2025-04-14",
    # "gpt-4.1-mini-2025-04-14",
    # "gpt-4.1-nano-2025-04-14", # even with retries, low approval rate in reasoning after
    # "gpt-4o-2024-11-20",
    # "gpt-4o-2024-08-06",
    # "gpt-4o-2024-05-13",
    # "gpt-4o-mini-2024-07-18",
    # "gpt-3.5-turbo-0125",
    # "gpt-3.5-turbo-1106",

    # "google/gemini-2.5-flash-preview", # we trigger the thinking mechanism in reasoning first
    # "google/gemini-2.0-flash-001",
    # "google/gemini-2.0-flash-lite-001",
    # "google/gemini-pro-1.5",
    # "google/gemini-flash-1.5",
    # "google/gemini-flash-1.5-8b",

    # "meta-llama/llama-4-maverick",
    # "meta-llama/llama-4-scout",
    # "meta-llama/llama-3.3-70b-instruct",
    # "meta-llama/llama-3.1-70b-instruct",
    # "meta-llama/llama-3.1-8b-instruct",
    # "meta-llama/llama-3-70b-instruct",
    # "meta-llama/llama-3-8b-instruct",

    # "anthropic/claude-3.5-sonnet-20240620:beta",
    # "anthropic/claude-3.5-haiku:beta",

    # "qwen/qwen-turbo",
    # "qwen/qwen-plus",
    # "qwen/qwen-max",
    # "qwen/qwen-2.5-72b-instruct",
    # "qwen/qwen-2.5-7b-instruct",

    # "qwen/qwen-2-72b-instruct",
    # "google/gemma-2-27b-it",
    # "google/gemma-2-9b-it",

    # "Qwen2.5-1.5B-Instruct",
    # "Qwen2.5-3B-Instruct",
    # "Qwen2.5-14B-Instruct",
    # "Qwen2.5-32B-Instruct",

    # "Qwen1.5-4B-Chat",
    # "Qwen1.5-7B-Chat",
    # "Qwen1.5-14B-Chat",
    # "Qwen1.5-32B-Chat",
    # "Qwen1.5-72B-Chat",

    "qwen/qwen3-32b:nothink",

    # "deepseek/deepseek-chat",
    # "deepseek/deepseek-chat-v3-0324",

    # "microsoft/phi-4",

    # "google/gemma-3-4b-it",
    # "google/gemma-3-12b-it",
    # "google/gemma-3-27b-it",

    # "mistralai/mixtral-8x7b-instruct", # low approval rate
    # "mistralai/mistral-7b-instruct-v0.3",
    # "mistralai/mistral-7b-instruct-v0.1",
    # "mistralai/mistral-nemo",
    # "mistralai/mistral-small",
    # "mistralai/mistral-large",
    # "mistralai/mistral-large-2407",

    # "amazon/nova-micro-v1",
    # "amazon/nova-lite-v1",
    # "amazon/nova-pro-v1",


    # # # RQ2 # # #
    # "OLMo-2-0325-32B-SFT",
    # "OLMo-2-0325-32B-DPO",

    # "OLMo-2-1124-13B-SFT",
    # "OLMo-2-1124-13B-DPO",
]
judge_model_ids = [
    # "gpt-4.1-mini-2025-04-14",
    # "gpt-4o-2024-08-06",
    "google/gemini-2.5-flash-preview",
    # "gemini-2.5-flash-preview-04-17",
]
RESULTS_DIR = "data/20250507/reasoning_after_runs/"
# RESULTS_DIR = "data/20250507/dpo_after_runs/"

In [ ]:
exps = [
    "after1",
    "after2",
    "after3",
    "after4",
    "after5",
]
for decision_model_id in decision_model_ids:
    continue_to_next_model = False
    for exp in exps:
        dr = DilemmaRunner(
            model_id=decision_model_id,
            decision_run_name=exp,
            results_dir=RESULTS_DIR,
            override_decision_temperature=1.0,  # default 0.0
            prompts_template='reasoning_after',
            # prompts_template='no_reasoning',
            choices_filename="choices_672.csv",
        )
        await dr.run(
            # limit=10,
            # disable_validation=True,
            # overwrite=True,
            try_retries=False,
        )
        # dr.process()
    print(f"Finished decision model {decision_model_id}.\n\n")


In [ ]:
exps = [
    "after1",
    "after2",
    "after3",
    "after4",
    "after5",
]
for decision_model_id in decision_model_ids:
    for exp in exps:
        jr = JudgeRunner(
            decision_model_id=decision_model_id,
            decision_run_name=exp,
            judge_model_id=judge_model_ids[0],
            judge_run_name=exp,
            results_dir=RESULTS_DIR,
            judge_cot=True,
            override_judge_temperature=0.0,
        )
        await jr.run_rationales()

    print(f"Finished analyzing decision model {decision_model_id}.\n\n")